In [4]:
!pip install matplotlib

In [23]:
import pandas as pd
from datetime import timedelta
import matplotlib.pyplot as plt
import numpy as np

# Configurar las fechas deseadas para el filtrado de los datos en formato 'YYYY-MM-DD'
initial_date = '2024-05-01'
end_date = '2024-05-31'

# Configurar las horas deseadas para el filtrado de los datos en formato 'HH:MM:SS' (Horas Laborales en este caso)
initial_hour = pd.to_datetime('08:00:00').hour
end_hour = pd.to_datetime('18:59:00').hour

# Configurar la ubicacion del archivo CSV con los reportes generados
path_file = 'C:/Users/crisb/PycharmProjects/ETL_SECURITY/Reports/Reportes Ticketera.csv'

# Se carga la data original. Para futuras modifcaciones 

df_original = pd.read_csv(path_file, sep=',')
df_original.head(10)

# Se procede a eliminar aquellos tickets cerrados con e tema de Notificaciones Bancarias
df = df_original.copy()
df = df[df['Temas de ayuda'] != 'RENOVACIONES - NOTIFICACION BANCARIA']

df.reset_index(drop=True, inplace=True)

# Filtrado desde la fecha inicial hasta la fecha  final deseada
df['Fecha de creación'] = pd.to_datetime(df['Fecha de creación'])
df['Fecha de cierre'] = pd.to_datetime(df['Fecha de cierre'])


df = df[df['Fecha de creación'] >= initial_date]
df = df[df['Fecha de creación'] <= end_date]
df = df.reset_index(drop=True)

# Filtrado de los tickets que fueron creados dentro del horario laboral (08:00 - 18:59)
df_in_range = df[df['Fecha de creación'].dt.hour > initial_hour]
df_in_range = df_in_range[df_in_range['Fecha de creación'].dt.hour < end_hour]

#Filtrado de los tickets que fueron cerrados en el mismo día de su creación
df_in_range = df_in_range[df_in_range['Fecha de cierre'].dt.date == df_in_range['Fecha de creación'].dt.date]

#Calculo del tiempo de respuesta de cada ticket
df_in_range['Time_Open_Ticket'] = df_in_range['Fecha de cierre'] - df_in_range['Fecha de creación']

# Funcion para asignar un valor de 1 a los tickets que no han sido reabiertos y un valor adicionado a 1 a los tickets que han sido reabiertos
def asignar_valor(contador):
    if pd.isnull(contador):
        return 1
    else:
        return contador + 1
    
# Funcion para asignar un valor de 1 a los tickets que no han sido reabiertos y un valor adicionado a 1 a los tickets que han sido reabiertos
df_in_range['Response by Ticket'] = df_in_range['Reabrir contador'].apply(lambda x: asignar_valor(x))

# Se agrupan los tickets por agente asignado y se calcula el total de respuestas y el tiempo total de respuesta
df_in_range['Total Responses'] = df_in_range.groupby('Agente asignado')['Response by Ticket'].transform('sum')
df_in_range['Total Time'] = df_in_range.groupby('Agente asignado')['Time_Open_Ticket'].transform('sum')

# Se calcula el tiempo promedio de respuesta de los tickets de renovaciones
df_in_range['Average Time'] = df_in_range['Total Time'] / df_in_range['Total Responses']
df_in_range['Average Time'] = df_in_range['Average Time'].apply(lambda x: timedelta(seconds=round(x.total_seconds())))
df_in_range['Average Time'] = df_in_range['Average Time'].apply(lambda x: str(x).split(' ')[-1])
df_final_report = df_in_range.groupby('Agente asignado').agg({'Total Responses': 'first', 'Total Time': 'first', 'Average Time':'first'}).reset_index()

df_final_report.sort_values(by='Average Time', ascending=False, inplace=True)
df_final_report.info()

print(df_final_report)

<class 'pandas.core.frame.DataFrame'>
Index: 7 entries, 0 to 4
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype          
---  ------           --------------  -----          
 0   Agente asignado  7 non-null      object         
 1   Total Responses  7 non-null      float64        
 2   Total Time       7 non-null      timedelta64[ns]
 3   Average Time     7 non-null      object         
dtypes: float64(1), object(2), timedelta64[ns](1)
memory usage: 280.0+ bytes
        Agente asignado  Total Responses      Total Time Average Time
0          Carlos Giler              9.0 0 days 18:07:13     02:00:48
1  Cristhian Benalcazar             13.0 0 days 14:52:57     01:08:41
2           Erika Duran             23.0 0 days 13:39:47     00:35:39
3       Ginella Alcivar             32.0 0 days 18:07:23     00:33:59
5          Jorge Asanza             38.0 0 days 20:41:57     00:32:41
6       Julissa Alfonzo            553.0 8 days 06:42:30     00:21:34
4     Guillerm

In [17]:
df_in_range['Average Time'].str.split(' ')

AttributeError: Can only use .str accessor with string values!

In [18]:
df_in_range['Average Time'].str

AttributeError: Can only use .str accessor with string values!

In [22]:
df_in_range['Average Time'].apply(lambda x: str(x).split(' ')[-1])

0      00:21:34
1      00:21:34
2      00:21:34
3      00:21:34
5      00:21:34
         ...   
634    00:21:34
635    00:21:34
636    00:21:34
637    00:21:34
638    00:21:34
Name: Average Time, Length: 467, dtype: object